The goal for this piece of code is:
- Clean the data for the following datasets:
    - K2
    - APOKSAC
    - APOGEE
    - GALAH

After we have cleaned the datas is
- Use K2 model to predict the mass for APOGEE and GALAH
- Use APOKSAC model to predict mass for APOGEE

The goal is to get a table contain the following for all the data sets and use this to predict the mass of stars.
The table include parameters such as:

- Effective Temperature
- Log g
- Iron Abundance [Fe/H]
- Alpha Abundance [alpha/Fe]
- Carbon Abundance [C/Fe]
- Nitrogen Abundance [N/Fe]
- Oxygen Abundance [O/Fe]

Importing the required packages

In [2]:
from astropy.table import Table
import pandas as pd         # For table/data management

# Cleaning the GALAH Data

In [2]:
# Import the data and change into a data frame
galah = Table.read("Data/galah_dr4_allspec_220713.fits", format = "fits")
names = [name for name in galah.colnames if len(galah[name].shape) <= 1]
galah = galah[names].to_pandas()

For Apogee Data, there are a few recommendations (this usually indicates that the data has some issues, e.g. unreliably broadening and other issues) for the flag that should be applied:
- `snr_c3_iraf > 30`
- `flag_sp == 0`
- For any element `X`, the flag `flag_X_fe == 0`

More information can be found [the GALAH Survey Website](https://www.galah-survey.org/dr3/flags/)

In [ ]:
# Filter the data
# galah = galah.loc[galah["snr_c3_iraf"] > 30]
galah = galah.loc[galah["flag_sp"] == 0]
galah = galah.loc[galah["flag_fe_h"] == 0]
galah = galah.loc[galah["flag_al_fe"] == 0]
galah = galah.loc[galah["flag_c_fe"] == 0]
galah = galah.loc[galah["flag_n_fe"] == 0]
galah = galah.loc[galah["flag_o_fe"] == 0]

Now the appropriate flag has been used, we extract only the useful columns and save the data for later.

In [3]:
galah = galah[["teff", "logg", "fe_h", "al_fe", "c_fe", "n_fe", "o_fe"]]
galah.to_csv("Data/cleaned_galah.csv", index=False)

# Cleaning APOGEE Data

In [7]:
apogee = Table.read("Data/allStar-dr17-synspec_rev1.fits", format="fits", hdu=1)
names = [name for name in apogee.colnames if len(apogee[name].shape) <= 1]
apogee = apogee[names].to_pandas()

For APOGEE, we do something similar by making sure all the parameters are working on are good.
More information can be found [SDSS explanantion of bitmasks](https://www.sdss.org/dr13/algorithms/bitmasks/#APOGEE_ASPCAPFLAG) and [SDSS recommendations for bitmasks](https://www.sdss.org/dr13/irspec/abundances/)

The parameters
- `ASPCAPFLAG & STAR_BAD == 0` (`STAR_BAD` is the 23rd bit (starting from 0), or  `STAR_BAD = 0b 1000 0000 0000 0000 0000 0000`)
    - `STAR_BAD` is a general indication of whether there are issues with measuring surface temperature, surface gravity, and check if any elements have `GRIDEDGE_BAD` set
- `ELEMFLAG`
    - Any example of this is `FE_H_FLAG`
        - Make sure that the flag does not contain `GRIDEDGE_BAD`, `GRIDEDGE_WARN` and `CALRANGE_BAD`
    - We should just check that the flag is equal to zero to eliminate all and any problematic behaviour

In [8]:
# Filter the data
STAR_BAD = 1 << 23
apogee = apogee.loc[apogee['ASPCAPFLAG'] & STAR_BAD == 0]
apogee = apogee.loc[apogee['FE_H_FLAG'] == 0]
apogee = apogee.loc[apogee['C_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['AL_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['N_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['O_FE_FLAG'] == 0]

Extract the useful column and save for later

In [10]:
apogee = apogee[["TEFF", "LOGG", "FE_H", "AL_FE", "C_FE", "N_FE", "O_FE"]]
apogee.columns = apogee.columns.str.lower()
apogee.to_csv("Data/cleaned_apogee.csv", index=False)

# Cleaning APOKSAC 2 Data
TODO: Not sure how to do this


# Cleaning K2 Data
TODO: Not sure how to do this

In [33]:
k2_cols = ["EPIC", "kappa-r-c", "e_kappa_r-c", "kappa-m-c", "e_kappa-m-c", "kappa-r-A2Z", "kappa-r-c-BAM", "kappa-r-c-BHM", "kappa-r-c-CAN", "kappa-r-c-SYD", "e_kappa-r-c-A2Z", "e_kappa-r-c-BAM", "e_kappa-r-c-BHM", "e_kappa-r-c-CAN", "e_kappa-r-c-COR", "e_kappa-r-c-SYD", "kappa-m_A2Z", "kappa-m-c-BAM", "kappa-m-c-BHM", "kappa-m-c-CAN", "kappa-m-c-COR", "kappa-m-c-SYD", "e_kappa-m_A2Z", "e_kappa-m-c-BAM", "e_kappa-m-c-BHM", "e_kappa-m-c-CAN", "e_kappa-m-c-COR", "e_kappa-m-c-SYD"]

k2_col_width = [
    (1, 9), (11, 16), (18, 23), (25, 29), (18, 23),
    (25, 29), (31, 35), (37, 42), (44, 49), (51, 56),
    (58, 63), (65, 70), (72, 77), (79, 84), (86, 91),
    (93, 98), (100, 104), (106, 110), (112, 118), (120, 124),
    (126, 130), (137, 143), (145, 149), (151, 155), (157, 162),
    (164, 169), (171, 175), (177, 181), (183, 187), (189, 194)
]
k2 = pd.read_fwf("Data/k2_mass_coefficient_data.txt", colspecs=k2_col_width)
k2 = pd.read_fwf("Data/k2_mass_coefficient_data.txt")
k2

,201051625,14.757,2.215 1.324 0.404 27.176,16.728 13.100 16.569,3.174,3.066 0.976 0.897,4.366,1.673 1.034 1.645,1.131,0.616 0.160 0.207
0,201079541,10.610,0.316 1.523 0.102 13.809 11.093,10.849 10.801,0.651 0.348,0.318 0.425,2.552 1.608,1.555 1.563,0.341 0.113,0.105 0.148
1,201082897,9.833,0.440 1.027 0.095 12.600 9.651,10.477 9.545 10.089 10.188,0.727 0.612,1.053 0.262 0.450 0.381,1.654 0.976,1.160 0.975 1.065 1.089,0.259 0.130,0.238 0.075 0.116 0.104
2,201084449,12.473,0.440 1.544 0.123 14.888 12.275,13.176 13.001,1.740 0.326,1.871 0.561,2.196 1.446,1.682 1.657,0.560 0.098,0.481 0.171
3,201084664,14.002,1.691 1.076 0.272 22.506,15.334 13.586,3.335,0.816 0.837,2.788,1.230 1.006,0.866,0.161 0.146
4,201085409,17.686,0.545 1.617 0.151 27.418,18.412 18.777,2.321,1.152 2.252,3.875,1.737 1.831,0.885,0.246 0.462
...,...,...,...,...,...,...,...,...,...,...
18816,251619708,9.697,0.377 0.880 0.076 19.116,9.791 10.464 9.579,1.691,0.567 0.529 0.543,3.385,0.903 1.037 0.845,0.815,0.111 0.120 0.114
18817,251620873,9.329,0.471 0.663 0.102,9.734 9.536,NaN,0.999 0.550,NaN,0.716 0.674,NaN,0.164 0.094
18818,251621333,6.843,0.123 1.193 0.046,7.134 7.023,NaN,0.317 0.271,NaN,1.270 1.241,NaN,0.134 0.113
18819,251622742,11.998,0.927 1.497 0.248 12.233,13.303 10.914 12.845 12.671,0.836,1.526 1.129 0.615 0.602,1.561,1.821 1.180 1.680 1.674,0.265,0.426 0.284 0.187 0.188
